# 생물정보학 및 실습 1 - Free Project
## LIN28A binding motif predictor 만들기 

### 환경 세팅 하기 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


 Bioconda 환경 설치


In [2]:
!git clone https://github.com/hyeshik/colab-biolab.git
!cd colab-biolab && bash tools/setup.sh
exec(open('colab-biolab/tools/activate_conda.py').read())

Cloning into 'colab-biolab'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 69 (delta 22), reused 54 (delta 13), pack-reused 0
Unpacking objects: 100% (69/69), done.
./
./root/
./root/.vimrc
./root/.bashrc.biolab
./root/.bin.priority/
./root/.bin.priority/pip3
./root/.bin.priority/pip
./root/.bin.priority/pip2
./root/.tmux.conf
./root/.condarc
./root/.profile
--2022-05-27 05:24:52--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76607678 (73M) [application/x-sh]
Saving to: ‘miniconda3.sh’

miniconda3.sh       100%[===================>]  73.06M  85.9MB/s    in 0.9s    

2022-05-27 05:24:53 (85.9 MB/s) - ‘miniconda3.sh’ sav

In [3]:
!conda install -y bedtools bioawk

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /root/conda/envs/lab

  added / updated specs:
    - bedtools
    - bioawk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bioawk-1.0                 |       h7132678_7         196 KB  bioconda
    ------------------------------------------------------------
                                           Total:         196 KB

The following NEW packages will be INSTALLED:

  bioawk             bioconda/linux-64::bioawk-1.0-h7132678_7



bioawk-1.0           | 196 KB    | : 100% 1.0/1 [00:00<00:00,  3.54it/s]                
Preparing 

In [4]:
%cd /content/drive/MyDrive/binfo1-datapack01/

/content/drive/MyDrive/binfo1-datapack01


### Fig SC3 그리기 

전체 Genome에 대해서 pileup 해주기 

In [5]:
!samtools mpileup CLIP-35L33G.bam > CLIP-35L33G.pileup
!wc -l CLIP-35L33G.pileup

[mpileup] 1 samples in 1 input files
959697300 CLIP-35L33G.pileup


In [6]:
!head CLIP-35L33G.pileup

chr1	3056473	N	4	^!G^!G^!G^!G	HGGI
chr1	3056474	N	4	AAAA	HB@I
chr1	3056475	N	4	AAAA	GG?I
chr1	3056476	N	4	TTTT	HGGH
chr1	3056477	N	4	GGGG	HEGI
chr1	3056478	N	4	GGGG	GEEI
chr1	3056479	N	4	AAAA	EA=I
chr1	3056480	N	4	AAAA	@AEH
chr1	3056481	N	4	GGGG	GFFI
chr1	3056482	N	4	TTTT	G=BG


In [7]:
!tail CLIP-35L33G.pileup

MU069435.1	31102	N	4	GGGG	IDDB
MU069435.1	31103	N	4	GGGG	DEBE
MU069435.1	31104	N	4	GGGG	EDDE
MU069435.1	31105	N	4	TTTT	@><8
MU069435.1	31106	N	4	TTT$T$	DG<;
MU069435.1	31107	N	2	GG	BC
MU069435.1	31108	N	2	GG	B>
MU069435.1	31109	N	2	TT	@3
MU069435.1	31110	N	2	GG	B1
MU069435.1	31111	N	2	G$G$	@6


Filtering with depth (30)

In [ ]:
!awk '$4 > 30 {print $0;}' CLIP-35L33G.pileup > CLIP-35L33G-filt.pileup
!wc -l CLIP-35L33G-filt.pileup

Match와 Substitution만 남기기 

In [ ]:
import pandas as pd 
import re 
import collections 
import math

def estimate_shannon_entropy(rna_sequence):
    m = len(rna_sequence)
    bases = collections.Counter([tmp_base for tmp_base in rna_sequence])
 
    shannon_entropy_value = 0
    for base in bases:
        # number of residues
        n_i = bases[base]
        # n_i (# residues type i) / M (# residues in column)
        p_i = n_i / float(m)
        entropy_i = p_i * (math.log(p_i, 2))
        shannon_entropy_value += entropy_i
 
    return shannon_entropy_value * -1


# chunck 
pileup = pd.read_csv('')


def chunk_processing(chunck): # define a function that you will use on chunks
    ## Do Something # your function code here
    toremove = re.compile('[<>$*#^]')
    chunck['matches'] = chunck['basereads'].apply(lambda x: toremove.sub('', x))
    chunck['entropy'] = chunck.apply(lambda x: estimate_shannon_entropy(x['matches']), axis=1)
    return chunck


chunk_list = [] # create an empty list to hold chunks
chunksize = 10 ** 6 # set chunk size
for chunk in pd.read_csv('CLIP-35L33G-filt.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'],chunksize=chunksize): # read in csv in chunks of chunksize
    processed_chunk = chunk_processing(chunk) # process the chunks with chunk_processing() function
    chunk_list.append(processed_chunk) # append the chunks to a list
df_concat = pd.concat(chunk_list) # concatenate the list to a dataframe

Filtering with Entropy (0.7) 

In [ ]:
filt_entropy = df_concat[df_concat['entropy'] >= 0.7].copy()
filt_entropy.sort_values(by='entropy', ascending=False).nlargest(1000, 'entropy')

In [5]:
!samtools view CLIP-35L33G.bam | less

SRR458758.23028115      0       chr1    3056473 0       20M     *       0        0       GAATGGAAGTTCAAGGATCT    HHGHHGE@GGHHED?GEGDG    MD:Z:20 NH:i:40 HI:i:1   NM:i:0  SM:i:0  XQ:i:40 X2:i:40 XO:Z:UM XS:A:-
SRR458758.23879202      0       chr1    3056473 0       20M     *       0        0       GAATGGAAGTTCAAGGATCT    GBGGEEAAF=CEEDEGBGG>    MD:Z:20 NH:i:40 HI:i:1   NM:i:0  SM:i:0  XQ:i:40 X2:i:40 XO:Z:UM XS:A:-
SRR458758.23893532      0       chr1    3056473 0       20M     *       0        0       GAATGGAAGTTCAAGGATCT    G@?GGE=EFBDDGGGGD@GG    MD:Z:20 NH:i:40 HI:i:1   NM:i:0  SM:i:0  XQ:i:40 X2:i:40 XO:Z:UM XS:A:-
SRR458758.26501547      0       chr1    3056473 0       20M     *       0        0       GAATGGAAGTTCAAGGATCT    IIIHIIIHIGHIIIHEHIHH    MD:Z:20 NH:i:40 HI:i:1   NM:i:0  SM:i:0  XQ:i:40 X2:i:40 XO:Z:UM XS:A:-
SRR458758.685997        16      chr1    3059006 0       22M     *       0        0       TTCATTTACAGAATGGAATACT  EG7:776DGGGBD<GGEBGEG@  MD:Z:22 NH:i:30 HI

sorting 결과에서 앞뒤로 각 10nt씩 21nt단위로 fasta file 만들기 


In [ ]:
!samtools fasta CLIP-35L33G.bam > CLIP-35L33G.fasta 

In [ ]:
from Bio import SeqIO 

with open("CLIP-35L33G.fasta", "r") as handle:
  seqs = SeqIO.parse(handle, "fasta")
  ## position - 10 to position + 10 
  SeqIO.write(xlinked, "xlinked.fasta", "fasta")

WebLogo Result 

hexamer 추출하기 

Training Model - CNN 